## Lesson 10 - Text Analytics & Word2Vec




### Table of Contents
* [電影劇本 Scripts analytics](#Movie-Script-Analytics)
* [word2vec挖掘語義相似關係](#word2vec-movie-scripts)
* [Load text file in folder to train Word2Vec model](#Load-text-file-in-folder-to-train-Word2Vec-model)
* [回到電影劇本看人物Word2Vec的關聯性](#Back-to-Movie-Script-Word2Vec)
* [Word2Vec調整參數](#Word2Vec-parameters)
* [Train Beauty class Word2Vec model](#Beauty-Word2Vec-model)
* [Recursively search Word2Vec words](#Recursively-search-Word2Vec-words)


<a id="Movie-Script-Analytics"></a>
## 電影劇本 Scripts analytics

流浪地球 The Wandering Earth
改編自劉慈欣中短篇科幻小說作品。郭帆執導，吳京主演，描述中國人拯救世界的科幻電影，敘述太陽極速衰老膨脹，地球面臨被吞沒的可怕災難。為拯救地球，人類在地球表面上建造了上萬座行星發動機，以求逃離太陽系。

<div>
    <img src="images/movie_cover.jpg" width="400">
    <center>Copyright & Disclaimer: All Pictures shown are for illustration purpose only</center>
</div>


In [ ]:
# 流浪地球劇本來源：https://www.bianju.me/Art_list.asp?ID=21729&page=18&CType=content
import re
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import jieba

import matplotlib.font_manager as fm
font_yahei_consolas = fm.FontProperties(fname='C:\Windows\Fonts\kaiu.ttf',size=14)
from matplotlib.font_manager import FontProperties
myfont=FontProperties(fname=r'C:\Windows\Fonts\kaiu.ttf',size=14)

%matplotlib inline

def cut_document_in_list(text_list, stopwords, exception):
    res_list = []
    punct = set(u''':!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
    punct |= set(exception)
    for i in text_list:
        # 以句號分句
        w1 = i.split("。")
        for j in w1:
            text = re.sub(r'，|。|？|：|“|”|！','',j.strip()) #刪除冊書符號
            # w1 = re.sub(name1,name2,w1) #對齊
            # w3 = list(jieba.cut(w2))#分詞
            # w4 = [w for w in w3 if w not in stopword] #去掉停用詞
            w4 = [word for word in jieba.cut(text, cut_all=False) if (len(word.strip()) >= 2) and (word not in stopwords) and ( not any(ext in word for ext in punct) )]
            outstr = ''
            for word in w4:
                outstr +=word
                outstr +=' '
            if len(outstr.strip())>1:
                res_list.append(outstr.strip())
    return res_list

stopwords = set()
with open('dict/stopwords.txt', 'r', encoding="utf8") as file:
    stopwords = file.readlines()
    stopwords = [stopword.strip('\n').strip() for stopword in stopwords]
except_file = open("dict/hippo_exception_word.txt", encoding='utf-8')
exception = except_file.read().split(',')
exception.append(" ")

with open('dict/movie_cast_name.txt',encoding= 'utf-8') as f1:
    data1 = f1.readlines()
with open('data/scripts/流浪地球.txt',encoding= 'utf-8') as f2:
    data2 = f2.readlines()
data2 = [x.replace('\n', '') for x in data2 if (len(x.strip()) >1)]
with open('data/scripts/流浪地球.txt',encoding= 'utf-8') as f2:
    data3 = f2.read()

In [ ]:
jieba.initialize()
jieba.load_userdict('dict/mydic2.txt')

r = cut_document_in_list(data2, stopwords, exception)
r[100:110]

In [ ]:
# 詞典裡的名字和劇本內容裡名字出現的次數
count = []
for name in data1:
    #count.append([name.strip(), data3.count(name.strip())])
    count.append([name.strip(), " ".join(r).count(name.strip())])
count1 = []
for i in count:
    if i not in count1:
        count1.append(i)
count = count1

count.sort(key = lambda x:x[1])
ay,ax = plt.subplots()
numbers = [x[1] for x in count[-10:]]
names = [x[0] for x in count[-10:]]
ax.barh(range(10),numbers,color=['peru','coral'],align = 'center')
ax.set_title('人物出場次數',fontsize = 14, fontproperties = font_yahei_consolas)
ax.set_yticks(range(10))
ax.set_yticklabels(names,fontsize = 14, fontproperties = font_yahei_consolas)
plt.show()

### 人物出場次數 與社交人脈網路關係圖
再來看看劇中人物的社交關係情況，採用以句為單位來進行分析（有時候也以段落為單位來識別人物關係，但採集的文本每集只有一個段落，所以不適用），即如果兩個人同時出現在一句話中，那說明他們之間肯定有某種關係。因此可以得到他們的社交網路關係。通過求得的共現矩陣，使用 Gephi 畫出下面的社交網路關係圖，圖中邊的粗細代表關係的密切程度，邊越粗表示兩人的關係越密切，而名字的大小可以表示為該人的社交人脈強弱情況。

In [ ]:
#匹配詞典裡的名字和劇本內容裡的該名字出現的次數
count = []
for name in data1:
    count.append([name.strip('\n'), " ".join(r).count(name.strip('\n'))])
count1 = []
for i in count:
    if i not in count1:
        count1.append(i)
count = count1

count.sort(key = lambda x:x[1])
ay,ax = plt.subplots()
numbers = [x[1] for x in count[-10:]]
names = [x[0].strip('\n') for x in count[-10:]]
ax.barh(range(10),numbers,color=['peru','coral'],align = 'center')
ax.set_title('人物出場次數',fontsize = 14,fontproperties = font_yahei_consolas)
ax.set_yticks(range(10))
ax.set_yticklabels(names,fontsize = 14,fontproperties = font_yahei_consolas)

#社交網路關係（共現矩陣）
word = data2
name = data1
#name = data1[1:]
#總人數
wordcount = len(name) 

#初始化128*128值全為0的共現矩陣
cormatrix = [[0 for col in range(wordcount)] for row in range(wordcount)] 
#遍歷矩陣行和列  
for colindex in range(wordcount):
    for rowindex in range(wordcount):
        cornum = 0
        #如果兩個人名字在同一句話裡出現，那麼共現矩陣中兩個人對應的值加1
        for originline in word:
            if name[colindex].strip('\n') in originline and name[rowindex].strip('\n') in originline:
                cornum += 1
        cormatrix[colindex][rowindex] = cornum

cor_matrix = np.matrix(cormatrix)
for i in range(len(name)):
    cor_matrix[i,i] = 0
social_cor_matrix = pd.DataFrame(cor_matrix, index = name, columns = name)
#把共現矩陣存進excel
social_cor_matrix.to_csv('report/movie/social_cor_matrix.csv')
social_cor_matrix.head()

In [ ]:
social_contact = pd.DataFrame(columns = ['name1','name2','frequency'])
#共現頻率
for i in range(0,len(name)):
    for j in range(0,len(name)):
        if i<j and cormatrix[i][j] > 0:
            social_contact.loc[len(social_contact),'name1'] = name[i].strip('\n')
            social_contact.loc[len(social_contact)-1,'name2'] = name[j].strip('\n')
            social_contact.loc[len(social_contact)-1,'frequency'] = cormatrix[i][j]

social_contact.to_excel('report/movie/social_contact.xlsx',index = False)
social_contact.head()

![社交網路關係（共現矩陣）](images/movie_matrix_gephi.png)

### 被推走的 "地球" 共現文字雲

接下來重點探索一下大衛比較感興趣的 "地球" 為本劇的主角 (被推走?!)，先通過關鍵字抓取出提及的劇本，然後使用 wordcloud 產生文字雲，wordcloud可以導入圖片自訂詞雲的形狀，非常方便，但是需要注意中文編碼和字體的問題，否則生成的圖片會顯示成亂碼。

In [ ]:
with open('data/scripts/流浪地球.txt',encoding= 'utf-8') as f2:
    data2 = f2.readlines()
data2 = [x.replace('\n', '') for x in data2 if (len(x.strip()) >1)]
word = data2

In [ ]:
word[10:15]

In [ ]:
import numpy as np
from PIL import Image
from wordcloud import WordCloud
from matplotlib import pyplot as plt
%matplotlib inline

#地球（感興趣的關鍵字，可能需要加入到詞典中）
text = []
#遍歷每句話
for line in word:
    if '地球' in line:
        text.append(line)

#詞頻統計
dict_dz = {}
for i in text:
    dz1 = i.split(' ')
    for w in dz1:
        w1 = w.strip()
        if dict_dz.__contains__(w1) :
            dict_dz[w1] += 1
        else:
            dict_dz[w1] = 1     
                            
#生成text
text1 = ''
for i in text:
    dz2 = i.split(' ')
    for w in dz2:
        text1 =text1 +' '+ w
                
#產生文字雲
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator

# 設置字體格式路徑以顯示中文
# 可以改成 'C:\Windows\Fonts\xxxx.ttf' 來切換其他字體，在此大衛將ttf檔放在預設資料夾中
# 選擇已經有的字體，根據詞頻否則生成圖片的時候會報錯：OSError: cannot open resource

#讀取背景圖片資訊保存為array
alice_mask = np.array(Image.open("images/cloud_mask7.png"))
font = 'font/simsun.ttf'
wc = WordCloud(background_color = 'white', mask = alice_mask,
               max_words = 2000, stopwords=STOPWORDS, font_path=font,
               max_font_size = 80, random_state = 42,scale = 1.5).generate(text1)

#store to file
picture_name = "images/movie_wordcloud.png"
wc.to_file(picture_name)

# show word cloud
#plt.rcParams["figure.figsize"] = (100,80)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.show()

詞雲中詞語越大代表和大風廠這個詞相關度越高，可以看出和地球關係最緊密的有：流浪地球、劉啟、領航員、劉培強，或者遠一些的韓子昂、三百年後等
回到前一session一開始提到的使用TF-IDF提取關鍵字，來看看電影劇本的結果。

In [ ]:
from jieba import analyse

tfidf = analyse.extract_tags
#analyse.set_stop_words('stop_words.txt') #使用自定義的停用詞

text_dz = ''
for l in text:
    text_dz += l
    text_dz += ' '
keywords = tfidf(text_dz,topK=20)
print (keywords)

### Homework
- choose your data set of movie script on internet
- draw co-relation graph using Gephi software
- draw a Word cloud for you to explore what is your finding?

<a id="word2vec-movie-scripts"></a>
## word2vec挖掘語義相似關係

文章分析中也經常會用到word2vec將文字轉化為詞向量的形式，用以挖掘詞彙在語句前後的相似關係。使用word2vec測試一下模型效果。導入gensim 後，將文章轉化為模型所需的輸入形式，即可開始進行訓練，產出每個詞彙的向量格式，訓練模型的定義：

![Word2Vec（詞向量）](images/Word2Vec.png)

在Word2Vec出來之前，我們常用的主要是one hot encoding的方法，也就是對於每一個單字進行編碼，即用在一個位置為1，其餘位置為0的向量來表示。
向量的維度就是我們單字量的大小，向量的每一個位置，只能用來表示唯一的一個單字。

舉個例子，假設我們的有詞庫只有10個單字，分別是:
比，特，幣，昨，天，就，開，始，大，漲。 以one hot encoding的方法來表示每一個詞，那麼有下面的結果:


比 → [1,0,0,0,0,0,0,0,0,0] <br>
特 → [0,1,0,0,0,0,0,0,0,0] <br> 
幣 → [0,0,1,0,0,0,0,0,0,0] <br>
昨 → [0,0,0,1,0,0,0,0,0,0] <br>
天 → [0,0,0,0,1,0,0,0,0,0] <br>
就 → [0,0,0,0,0,1,0,0,0,0] <br>
開 → [0,0,0,0,0,0,1,0,0,0] <br>
始 → [0,0,0,0,0,0,0,1,0,0] <br>
大 → [0,0,0,0,0,0,0,0,1,0] <br>
漲 → [0,0,0,0,0,0,0,0,0,1] <br>


可以看到對於每一個單字，唯一的一個向量對它進行了表示。那麼很顯然這種表示方法至少有下面的一些缺陷

1. 在常見的距離下面，單字與單字間的距離都是沒有差別的，"比"和"特"的距離和"比"和"漲"的距離是一樣的
2. 隨著單字量的增加，向量的維度也隨之增加，對於詞庫中沒有的新字，無法以唯一的向量一對一的mapping對映
3. 當單字量較多時，高維度向量對運算速度大大降低效率，無法適應於即時的訓練與作出調整

所以，字向量的提出目的就是解決上面提到的這些問題。而英文可能是用於Word(字)，中文則是要以詞彙(至少兩個字)的向量來做訓練。

1. 如果詞彙向量有N個，那麼可以用一個n維的向量來表示每一個詞彙，並且 n 遠遠小於N，常見的n為100到300維
2. 詞彙向量每一個位置不再是只能取0和1的數值，而是可以取任意的實數
3. 詞彙向量之間的差在一定程度上是有意義的，比如，台北的詞向量為 v1，台南的詞向量為 v2, 西雅圖的詞向量為 v3 , 紐約的詞向量為 v4 ，藉由word2vec學習出來的這些詞向量大致都有相符的特徵。這個近似關係，或是說城市關係，也正是受到word2vec的啟發，在知識圖譜表示學習中，衍生了一些名為Trans的編碼演算法
4. 除此之外，地名和地名在詞向量空間中的距離比地名和動物的詞向量距離近，等等，換句話說就是描述同一屬性和種類的詞向量的距離要小於不同屬性和種類的詞向量的距離

### word2vec 轉成高維空間向量 - 以字為單位

In [ ]:
from gensim.models import Word2Vec

sentences = '結果鄭大仁看到芷芷留言後贊聲：「免神經，咖小而已。」未料此舉引起連千毅的小弟西門慶的留言，請鄭男刪除留言，但芷芷卻又突然冒出：「有人被威脅要泡茶拜訪。」、「我想請問他怎麼不用道歉啊？每次都要逼人要來我家拜訪，這樣很不好耶，我沒有惹到他啊！」, 此時連的小弟西門慶又回覆指：「這是多久的事情了何必提？無風不起浪，別惹事，我想應該是他身邊的人提醒妳，他本人沒空理妳。」也有網友留言：「整天被害妄想，腦袋是有事？」, 但芷芷似乎不想罷手，還繼續留言：「我為何要當替死鬼？我真的好難過呀，難道無法溝通嗎？」沒想到此話一出，引起雙方隔空互嗆，西門慶再嗆鄭又仁：「我跟你不認識，你為了不知名的整容女子來嗆聲，你混很好的樣子？」鄭又仁不滿對方抬他老大名號出來反嗆：「咖小就是咖小，一年賺100億也他家的事，惹到不該惹的人。」 自封「亞洲直播天王」的連千毅，這陣子引爆高雄街頭大戰，這場「直播主之亂」頻頻登上社會版面，名字也總跟酒色財氣、黑道連在一起，私下已婚的他，後宮也陷入駁火狀態，本刊接獲讀者爆料，已婚的連千毅加上正妹老婆，1人共連尬4女就算了，還撒出大筆鈔票，讓好幾個爆乳辣妹為他撕逼撕得厲害，其中一名「小四」情婦還是40歲的單親媽媽，爭風吃醋的字句流彈四射，本刊取得連千毅安撫這名「小四」的私密對話，看得出來連千毅不管在直播或女人，舌功都很了得。打開連千毅的直播，總是跟香車、名錶、金錢有關，擅於炒作話題的他，也懂美女吸睛的道理，每次直播，身邊總是坐著一到兩個爆乳辣妹，但這些爆乳妹不僅僅只是直播來賓，也猶如已婚的連千毅後宮佳麗一般，備受他金援以及寵愛。'
model = Word2Vec(sentences, sg=1, size=100,  window=5,  min_count=1,  negative=3, sample=0.001, hs=1, workers=4)  

In [ ]:
len(model.wv.index2word)

In [ ]:
len(model.wv.index2word)

In [ ]:
# 檢視字詞
for i in range(9,20):
    print(model.wv.index2word[i])

In [ ]:
vector = model['西']
vector

In [ ]:
# 顯示空間和字距離相近的字
res = model.most_similar('西',topn = 100)  # 此字有出現在 corpus 當中
for item in res[:5]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

In [ ]:
# "word '西門' not in vocabulary"
vector = model['西門']

In [ ]:
# 計算兩字的相似度
model.similarity('西', '小')

### word2vec 轉成高維空間向量 - 以詞彙為單位

In [ ]:
import jieba

jieba.initialize()
jieba.load_userdict('dict/mydic.txt')

In [ ]:
def jiebaCut(x):
    punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
    # punct |= set(exception)
    words = [word for word in jieba.cut(x, cut_all=False) if (len(word.strip()) >= 2) and ( not any(ext in word for ext in punct) )]
    return words

sentences_list = []
sentences_list.append('結果鄭大仁看到芷芷留言後贊聲：「免神經，咖小而已。」未料此舉引起連千毅的小弟西門慶的留言，請鄭男刪除留言，但芷芷卻又突然冒出：「有人被威脅要泡茶拜訪。」、「我想請問他怎麼不用道歉啊？每次都要逼人要來我家拜訪，這樣很不好耶，我沒有惹到他啊！」, 此時連的小弟西門慶又回覆指：「這是多久的事情了何必提？無風不起浪，別惹事，我想應該是他身邊的人提醒妳，他本人沒空理妳。」也有網友留言：「整天被害妄想，腦袋是有事？」, 但芷芷似乎不想罷手，還繼續留言：「我為何要當替死鬼？我真的好難過呀，難道無法溝通嗎？」沒想到此話一出，引起雙方隔空互嗆，西門慶再嗆鄭又仁：「我跟你不認識，你為了不知名的整容女子來嗆聲，你混很好的樣子？」鄭又仁不滿對方抬他老大名號出來反嗆：「咖小就是咖小，一年賺100億也他家的事，惹到不該惹的人。」')
sentences_list.append('自封「亞洲直播天王」的連千毅，這陣子引爆高雄街頭大戰，這場「直播主之亂」頻頻登上社會版面，名字也總跟酒色財氣、黑道連在一起，私下已婚的他，後宮也陷入駁火狀態，本刊接獲讀者爆料，已婚的連千毅加上正妹老婆，1人共連尬4女就算了，還撒出大筆鈔票，讓好幾個爆乳辣妹為他撕逼撕得厲害，其中一名「小四」情婦還是40歲的單親媽媽，爭風吃醋的字句流彈四射，本刊取得連千毅安撫這名「小四」的私密對話，看得出來連千毅不管在直播或女人，舌功都很了得。打開連千毅的直播，總是跟香車、名錶、金錢有關，擅於炒作話題的他，也懂美女吸睛的道理，每次直播，身邊總是坐著一到兩個爆乳辣妹，但這些爆乳妹不僅僅只是直播來賓，也猶如已婚的連千毅後宮佳麗一般，備受他金援以及寵愛。')

sentences = list(map(jiebaCut, sentences_list))
model = Word2Vec(sentences, size=100,  window=5,  min_count=1,  negative=3, workers=4)

In [ ]:
len(model.wv.index2word)

In [ ]:
#文字以向量形式呈現
model.wv.vectors

In [ ]:
for x in model.wv.index2word[10:16]:
    print(x)

In [ ]:
# 顯示空間和字距離相近的詞彙
res = model.most_similar('連千毅',topn = 100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

In [ ]:
# 計算 兩詞彙 Cosine 相似度
res = model.similarity('連千毅', '正妹')
print(res)

In [ ]:
# 計算 三詞彙 Cosine 相似度
res = model.most_similar(['連千毅', '西門慶'], ['芷芷'], topn= 100)
print("%s之於%s，如%s之於: " % ('連千毅', '西門慶', '芷芷'))
print(res)

In [ ]:
for item in res[:10]:
    print("{}, {}".format(item[0], str(item[1])))

### Save news word2vec model

In [ ]:
modelname = "model/Word2Vec/word2vec_news.bin"
model.wv.save_word2vec_format(modelname, binary=True)

### Load news word2vec model

In [ ]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/word2vec_news.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [ ]:
# 測試 Word2Vec model
res = model.similarity('連千毅', '正妹')
print(res)

<a id="Load-text-file-in-folder-to-train-Word2Vec-model"></a>
## Load text file in folder to train Word2Vec model

In [ ]:
import os
import jieba
from gensim.models import Word2Vec

jieba.initialize()
jieba.load_userdict('dict/mydic.txt')

class Sentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
        jieba.load_userdict("dict/mydic.txt")

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
                # punct |= set(exception)
                words = [word for word in jieba.cut(line, cut_all=False) if (len(word.strip()) >= 2) and ( not any(ext in word for ext in punct) )]
                yield words

In [ ]:
# 歌詞
folder_path = "./data/lyrics/"
sentences = Sentences(folder_path)
model = Word2Vec(sentences, size=100,  window=5,  min_count=0,  negative=3, workers=4)

In [ ]:
model.wv.index2word[:20]

### Save lyrics Word2Vec Model

In [ ]:
modelname = "model/Word2Vec/word2vec_lyrics.bin"
model.wv.save_word2vec_format(modelname, binary=True)

### Load lyrics Word2Vec Model

In [ ]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/word2vec_lyrics.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [ ]:
model.wv.index2word

In [ ]:
# 測試訓練好的詞向量模型，使用 model[keyWord] 取出 keyword 這個詞彙的詞向量
keyWord = '眼淚'
print(model[keyWord])

如何評價所建「詞向量」的好壞？

依照 word2vec 的原理，詞意相近的詞在向量空間當中的距離是接近的，但會因為「文本內容性質的差異」而有所不同，例如在新聞類的文本中，「台灣」字詞會常與地名或時事事件等字詞距離接近；然而若是跟 PTT 論壇相關的文本，「台灣」字詞可能會更常與「鄉民/島民/魯蛇/溫拿」這類聊天用語的字詞距離靠近；總之，可以透過所訓練之詞向量距離，觀察他們語意是否相近去做衡量：

In [ ]:
# 顯示空間和字距離相近的關聯詞彙
res = model.most_similar('懷念',topn = 100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

In [ ]:
model.similarity('懷念', '那年')

<a id="Back-to-Movie-Script-Word2Vec"></a>
## 回到 電影流浪地球劇本 看人物 Word2Vec 的關聯性

In [ ]:
import os
import jieba
import pickle
from gensim.models import Word2Vec

jieba.initialize()
jieba.load_userdict('dict/movie_cast_name.txt')

stopwords = set()
with open('dict/stopwords.pkl', 'rb') as handle:
    stopwords = pickle.load(handle)

class Sentences(object):
    def __init__(self, dirname, stopwords):
        self.dirname = dirname
        jieba.load_userdict("dict/mydic.txt")

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
                # punct |= set(exception)
                words = [word for word in jieba.cut(line, cut_all=False) if (len(word.strip()) >= 2) and ( not any(ext in word for ext in punct) )]
                yield words

def cal_word2vec(word2vec_model, word, topn):
    try:
        vector = word2vec_model[word]
        res = model.most_similar(word, topn=topn)  # 此詞彙有出現在 corpus 當中
        return res
    except Exception as e:
        return "word '{}' not in vocabulary".format(word)

In [ ]:
# 電影
folder_path = "./data/scripts/"
sentences = Sentences(folder_path, stopwords)
model = Word2Vec(sentences, size=100,  window=5,  min_count=5,  negative=3, workers=4)

In [ ]:
model.wv.index2word[:10]

### Save movie Word2Vec Model

In [ ]:
modelname = "model/Word2Vec/word2vec_movie.bin"
model.wv.save_word2vec_format(modelname, binary=True)

### Load movie Word2Vec Model

In [ ]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/word2vec_movie.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [ ]:
print (model['地球'])

In [ ]:
keyword = "地球"
keyword = "劉啟"
res = cal_word2vec(model, keyword, 100)
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

<a id="Word2Vec-parameters"></a>
## Word2Vec 調整參數

```
from gensim.models import Word2Vec  
model = Word2Vec(sentences, sg=1, size=100,  window=5,  min_count=5,  negative=3, sample=0.001, hs=1, workers=4)  
```

In [ ]:
# 參數解釋：
# 其中的sentences是句子組成的 list，而每個句子又是詞彙的列表(非一句話或一篇文章)，即 list[list] 類型。

#1.sg=1是skip-gram演算法，對低頻詞敏感；預設sg=0為CBOW演算法。

#2.size是輸出詞向量的維數，值太小會導致詞映射因為衝突而影響結果，值太大則會耗記憶體並使演算法計算變慢，一般值取為100到200之間。

#3.window是句子中當前詞與目標詞之間的最大距離，3表示在目標詞前看3-b個詞，後面看b個詞（b在0-3之間隨機）。

#4.min_count是對詞進行過濾，詞頻小於 min-count 的單詞則會被忽略，預設值為5。

#5.negative和sample可根據訓練結果進行微調，sample表示更高頻率的詞被隨機下採樣到所設置的閾值，預設值為1e-3。

#6.hs=1展示層級softmax將會被使用，預設 hs=0 且 negative 不為 0，則負採樣將會被選擇使用。

#7.workers控制訓練的平行處理，此參數只有在Windows安裝了Cython或Linux環境才有效，否則只能使用單核，anaconda 會內建 Cython。

<a id="Beauty-Word2Vec-model"></a>
##  Train Beauty class Word2Vec model

In [1]:
import os
import jieba
import pickle
from gensim.models import Word2Vec

jieba.initialize()
jieba.set_dictionary("dict/dict.big.txt")

stopwords = set()
with open('dict/stopwords.pkl', 'rb') as handle:
    stopwords = pickle.load(handle)

class Sentences(object):
    def __init__(self, dirname, stopwords):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
                # punct |= set(exception)
                words = [word for word in jieba.cut(line, cut_all=False) if (len(word.strip()) >= 2) and (word not in stopwords) and ( not any(ext in word for ext in punct) )]
                yield words

def cal_word2vec(word2vec_model, word, topn):
    try:
        vector = word2vec_model[word]
        res = model.most_similar(word, topn=topn)  # 此詞彙有出現在 corpus 當中
        return res
    except Exception as e:
        return "word '{}' not in vocabulary".format(word)

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\princ\AppData\Local\Temp\jieba.cache
Loading model cost 0.813 seconds.
Prefix dict has been built succesfully.


### Train Beauty class model

In [2]:
# 美妝
folder_path = "./data/corpus/beauty_small/"
sentences = Sentences(folder_path, stopwords)
model = Word2Vec(sentences, size=240,  window=5,  min_count=5,  negative=3, workers=4)

Building prefix dict from D:\Programming\Python\課程教學\David Python 教育訓練\dict\dict.big.txt ...
Loading model from cache C:\Users\princ\AppData\Local\Temp\jieba.u9fe11b2227c896c5bbad65bb17730314.cache
Loading model cost 1.496 seconds.
Prefix dict has been built succesfully.


### Save Beauty class model

In [3]:
modelname = "model/Word2Vec/beauty_small.bin"
model.wv.save_word2vec_format(modelname, binary=True)

### Load Beauty class model

In [4]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/beauty_small.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [5]:
model.wv.index2word[:10]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


['彩妝', '唇膏', '肌膚', '活動', '保濕', '保養', '面膜', '眼影', '限量', '使用']

<a id="Recursively-search-Word2Vec-words"></a>
## Recursively search Word2Vec words

In [6]:
def gen_relation_keywords(model, query):
    try:
        list_new  = []
        layer_1 = list([])
        layer_2 = list([])
        res = model.most_similar(query, topn = 20)
        for item in res:
            similarity = item[1]
            if similarity > 0.5:
                child = item[0]
                layer_1.append(child)
                layer_2.append(child)

        for child in layer_1:
            res = model.most_similar(child, topn = 23)
            for item in res:
                similarity = item[1]
                if similarity > 0.5:
                    child = item[0]
                    if child not in layer_2:
                        layer_2.append(child)

            # Homework Hint:
            #for child in layer_2:
            #    res = model.most_similar(child, topn = 23)
            #    for item in res:
            #        similarity = item[1]
            #        if similarity > 0.5:
            #            child = item[0]

        print(len(layer_1))
        print(layer_1)
        print('='*20)
        print(len(layer_2))
        #print(layer_2)
        result_list = []
        for x in layer_2:
            result_list.append(x)
        print(result_list)

    except Exception as e:
        print('Exception:'+repr(e))
    else:
        pass
    finally:
        pass

In [7]:
query = '化妝'
gen_relation_keywords(model, query)

20
['肌膚', '唇膏', '使用', '彩妝', '底妝', '粉底', '眼影', '產品', '皮膚', '保養', '保濕', '面膜', '唇彩', '效果', '精華', '優惠', '限定', '商品', '秋冬', '系列']
40
['肌膚', '唇膏', '使用', '彩妝', '底妝', '粉底', '眼影', '產品', '皮膚', '保養', '保濕', '面膜', '唇彩', '效果', '精華', '優惠', '限定', '商品', '秋冬', '系列', '這次', '蜜粉', '修護', '品牌', '限量', '推出', '小編', '完美', '日本', '美妝', '聖誕', '包裝', '自然', '推薦', '粉底液', '護膚', '精華液', '適合', '活動', '週年']


### Homework
- 請實作加上第三層
- 加上第三層的關聯效果如何?